前面内容中，我们介绍了文本分类的基本流程和词袋模型，以及利用词袋模型对电影评论数据进行分类的案列。此部分，我们主要介绍主题模型在文本分类中的应用。 在主题模型中，主题表示一个概念、一个方面，表现为一系列相关的单词，是这些单词的条件概率。形象来说，主题就是一个桶，里面装了出现概率较高的单词，这些单词与这个主题有很强的相关性【引用自[CSDN博客](http://blog.csdn.net/huagong_adu/article/details/7937616)】。具体的理论阐述可参考[CSDN](http://blog.csdn.net/huagong_adu/article/details/7937616)和[知乎](https://www.zhihu.com/search?type=content&q=%E4%B8%BB%E9%A2%98%E6%A8%A1%E5%9E%8B)。

在本示例中，**我们主要介绍主题模型的基本思想和应用方法。**在前面的词袋模型中，我们将每篇文本直接表示成若干单词的频率向量，如：

| 句子 | I | like | he | likes | dogs | too| very | much
| - | :-: |  -: |  -: | -: |-: |-: |-: |-: |
| S1 | 1 | 1 | 0 | 0 | 1 | 0 | 1 | 1|
|S2 | 0 | 0 | 1 | 1 | 1 | 1 | 0 | 0|

在主题模型中，我们在文档和单词之间还引入了主题变量，也就是每篇文本我们可以看成是由若干主题组合而成，而各主题又拥有不同概率出现的单词。主题是一种隐变量，给我们任意一篇文章，我们只能看到文章以及里面的单词，看不到主题。但是可以建立文章和单词的矩阵，比如词袋模型中的频率矩阵，进行一定矩阵分解来获取主题，最后我们使用主题来表示文本。简单的可以将文档-主题，主题-单词用表格示意如下：

             文档-主题

| 句子 | 主题1 | 主题2 | 主题3 | 主题4 | 
| - | :-: |  -: |  -: | -: |
|文档1 | 1 | 1 | 0 | 0 | 
|文档2 | 0 | 0 | 1 | 1 | 

              主题-单词

| 主题 | 单词1 | 单词2 | 单词3 | 单词4 | 单词...
| - | :-: |  -: |  -: | -: |
|主题1 | 0 | 1 | 0 | 0 | ...|
|主题2 | 0 | 0 | 1 | 1 | ...|
|主题3 | 1 | 0 | 0 | 0 | ...|
|主题4 | 0 | 1 | 1 | 1 | ...|

我们可以把文章表示成主题以一定概率的组合而成的向量，而主题又可以表示成单词组合而成的向量，文档单词之间的条件概率可以分解成：
![topicmodel](http://img.my.csdn.net/uploads/201209/03/1346651721_6594.PNG)

图示方法：
![topicmodel](http://img.my.csdn.net/uploads/201209/03/1346651772_3109.PNG)


如果我们得到了文本的主题表示矩阵，可以利用主题向量进行分类。

## 1.文本数据预处理
电影评论数据可以直接通过nltk工具包直接获取，里面可以读取每篇评论内容、感情色彩以及单词。部分代码参考了CSDN博客http://blog.csdn.net/sinat_20791575/article/details/58661827 。

In [1]:
import nltk
import numpy as np
from nltk.corpus import movie_reviews

In [2]:
categories = movie_reviews.categories()
categories

['neg', 'pos']

电影评论从感情色彩上分为积极与负面两种类型

In [3]:
print('评论数量：', len(movie_reviews.fileids()))

评论数量： 2000


In [4]:
print('积极评论数量：', len(movie_reviews.fileids('pos')))
print('负面评论数量：', len(movie_reviews.fileids('neg')))

积极评论数量： 1000
负面评论数量： 1000


两种类型的数据是均衡的。下面我们查看第一个文件数据。

In [5]:
first = movie_reviews.fileids()[0]
movie_reviews.open(first).read()[:400]

'plot : two teen couples go to a church party , drink and then drive . \nthey get into an accident . \none of the guys dies , but his girlfriend continues to see him in her life , and has nightmares . \nwhat\'s the deal ? \nwatch the movie and " sorta " find out . . . \ncritique : a mind-fuck movie for the teen generation that touches on a very cool idea , but presents it in a very bad package . \nwhich i'

可以看到当前文档除了单词之外还有标点符号，数字换行符等。在后面的处理中，我们可以将这些标点符号看成是没有意义的去除掉，也可以保留下来作为一个符号进行分析，特别是问号、感叹号也包含一定感情色彩。

**注意，一般读取的文档内容都是句子或者段落的长文本，需要进行分词。**

这里我们保留所有的符号，下面我们通过nltk工具获取每篇文章的单词以及对应的情感标签。nltk已经对文本进行了分词，但是为了演示完整流程，我们使用原始的文本，将原始的长文本分成单词和标点符号序列；分词时还可以分成2元组、3元组(n-gram)，比如“machine learning"、“a good dog”。对于中文稍微有些复杂，因为没有空格来区分词组，比如“今天我买了一件衣服”，每个汉字可以当做一个词，但是根据常识应该分为“今天”、“我”、“买了”、“一件”、“衣服”。

In [6]:
#读取每篇评论及其对应的标签
document_label_pairs = [(movie_reviews.raw(fileid), category)
             for category in movie_reviews.categories()
             for fileid in movie_reviews.fileids(category)]


In [7]:
#随机打乱次序
np.random.shuffle(document_label_pairs)

In [12]:
documents, labels = list(zip(*document_label_pairs))

## 2.文本清洗

文本清洗可以看成是数据预处理。一般有两项工作：
- 去除无意义符号。文本一般还有文字、标点符号、空格、数字，甚至有很多缩写、外文、错误的拼写，有的时候我们只需要处理单纯的文本比如单词或汉字，其他符号可以当做噪声去除掉。
- 对文本进行分词。将文本内容分成词组等形式。

在上面我们已经利用nltk工具直接获得了每篇评论的单词序列，这里再自己做一遍。

如果原始文件是一个字符串，比如我们将上文看作一个长字符串，并且需要去掉标点符号，我们可以使用Python工具完成。

### 2.1文本清洗示例
可以正则表达式来去掉标点符号。

In [13]:
#正则表达，去除表达符号
import re
text = document_label_pairs[0][0]
#指定需要去掉的标点符号
text = re.sub('[,.!:;#()-]', '', text)
text[:400]

'according to the publicity material  with this movie the directors " hope to restore good oldfashioned bowling to its rightful place in the mainstream of american consciousness  " \nhmm  \nyou never know  they just might be on to something  \nwhat with the rise of geek chic  lounge music and seventies fashion  the evercontrary kids of the nineties might just latch on to bowling as another terminally '

In [14]:
#分词
print(text.split()[:20])

['according', 'to', 'the', 'publicity', 'material', 'with', 'this', 'movie', 'the', 'directors', '"', 'hope', 'to', 'restore', 'good', 'oldfashioned', 'bowling', 'to', 'its', 'rightful']


### 2.2对电影评论数据进行文本清洗

In [15]:
#去掉标点符号
documents = [re.sub('[_,.!:;#()0-9]', '', text) for text in documents]

### 2.3训练集和测试集划分



In [16]:
from sklearn.model_selection import train_test_split
doc_train, doc_test, label_train, label_test = train_test_split(documents, labels, test_size=0.3, random_state=111)

## 3.文本主题模型表达

文本数据特点是字符多、长度不一致，提取特征比一般的数据复杂。文本表达大致有三个方向，具体可参考[CSDN博客](http://blog.csdn.net/wangongxi/article/details/51591031)：
- 传统上的模型有词袋模型，也就是将文本看成单词的集合，不考虑单词的次序，比如"I love this dog"可以表达成[I, love, this, dog]，但是这样依然是文字符号，人能理解，计算机不能理解，所以需要转换成向量等形式以作为计算机的输入，比如将句子表示成词库中每个单词的频率组成的向量。通常文本库中的单词量很大，数万、数十万，很多单词只在部分文本中出现，所以文本表示的矩阵是维数很高的稀疏矩阵。
- 90年代有学者提出了[主题模型](http://blog.csdn.net/huagong_adu/article/details/7937616)，也就是将文档看成是若干主题的组合，比如一篇文章既讨论政治又讨论经济，而主题又是单词的组合。比较主流的有LSA方法，LDA方法等。
![Topic Model](http://img.my.csdn.net/uploads/201209/03/1346651772_3109.PNG)
- 近年来，随着深度学习的雄起，词嵌入（word embedding), 句子嵌入（sentence embedding）等概念和方法在文本处理中取得了突飞猛进。其理念是使用嵌入低维向量来表示单词、句子甚至文档，意义相近的单词距离较近，同理，意义相近的句子或文档向量距离也比较近。

**本次主要讨论主题模型**。

首先要通过词袋模型建立文档单词矩阵：


### 3.1电影评论频率特征提取
利用sklearn工具包进行特征提取，去掉连接词，最低保留单词频率为2。

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=2, stop_words='english', ngram_range=(1,1))

In [19]:
vectorizer.fit(doc_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=2,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [20]:
doc_train_vec = vectorizer.transform(doc_train)
doc_test_vec = vectorizer.transform(doc_test)

In [21]:
print('Training data Shape:', doc_train_vec.shape)
print('Testing data Shape:', doc_test_vec.shape)

Training data Shape: (1400, 19743)
Testing data Shape: (600, 19743)


In [22]:
import pandas as pd
data = pd.DataFrame(doc_train_vec.toarray())

In [23]:
data.columns = vectorizer.get_feature_names()
data.index = ['doc'+str(i+1) for i in range(len(data))]
data.head()

,aa,aardman,aaron,abandon,abandoned,abandonment,abandons,abby,abc,abducted,...,zoo,zoolander,zoologist,zoom,zooming,zooms,zoot,zorro,zucker,zwick
doc1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
doc5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


这是个很庞大的稀疏矩阵，所以sklearn将之存储到稀疏矩阵里。

### 3.2获取主题变量
我们尝试使用“Non-negative Matrix Factorization”和“Latent Dirichlet Allocation”两种方法进行主题提取。

In [44]:
from sklearn.decomposition import NMF, LatentDirichletAllocation
from time import time
n_components = 30
# Fit the NMF model, suppose there are 10 topics

nmf = NMF(n_components=n_components, random_state=1,
          alpha=.1, l1_ratio=.5)
lda = LatentDirichletAllocation(n_components =n_components, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
t0 = time()
nmf.fit(doc_train_vec)
print("NMF done in %0.3fs." % (time() - t0))
t0 = time()
lda.fit(doc_train_vec)
print("LDA done in %0.3fs." % (time() - t0))

NMF done in 11.055s.
LDA done in 7.299s.


In [29]:
feature_words = vectorizer.get_feature_names()

我们可以查看NMF方法提取到的每个主题里面包含哪些单词。

In [45]:
for i, component in enumerate(nmf.components_):
    #The most important 20 words' index
    nlargest = component.argsort()[-10:]
    word_list = []
    for n in nlargest:
        word_list.append(feature_words[n])
    print('*'*20)
    print('The ' + str(i) + 'th component:')
    print(word_list)

********************
The 0th component:
['fact', 'point', 'performance', 'scenes', 'really', 'plot', 'great', 'seen', 'films', 'film']
********************
The 1th component:
['watch', 'hollywood', 'interesting', 'think', 'seen', 've', 'minutes', 'plot', 'movies', 'movie']
********************
The 2th component:
['father', 'old', 'dog', 'year', 'love', 'time', 'world', 'family', 'man', 'life']
********************
The 3th component:
['times', 'things', 'harry', 'years', 'make', 'stories', 'doesn', 'love', 'characters', 'story']
********************
The 4th component:
['trilogy', 'characters', 'original', 'lucas', 'jar', 'jedi', 'phantom', 'menace', 'wars', 'star']
********************
The 5th component:
['fflewdurr', 'studio', 'video', 'film', 'disney', 'hen', 'wen', 'king', 'black', 'cauldron']
********************
The 6th component:
['course', 'series', 'planet', 'species', 'quite', 'films', 'fincher', 'ripley', 'aliens', 'alien']
********************
The 7th component:
['arnold', 'n

LDA方法提取的主题，有些主题里面单词比较相关。

In [46]:
for i, component in enumerate(lda.components_):
    #The most important 20 words' index
    nlargest = component.argsort()[-10:]
    word_list = []
    for n in nlargest:
        word_list.append(feature_words[n])
    print('*'*20)
    print('The ' + str(i) + 'th component:')
    print(word_list)

********************
The 0th component:
['daryl', 'private', 'investigator', 'film', 'arlo', 'pullman', 'gloria', 'stark', 'kasdan', 'zero']
********************
The 1th component:
['scene', 'trek', 'love', 'story', 'violins', 'time', 'just', 'movie', 'like', 'film']
********************
The 2th component:
['platitudes', 'cropped', 'overseen', 'hallucinatory', 'arc', 'worldview', 'wail', 'banshee', 'joan', 'besson']
********************
The 3th component:
['sexual', 'harry', 'character', 'time', 'just', 'like', 'movie', 'brody', 'film', 'ritchie']
********************
The 4th component:
['french', 'rodman', 'close', 'stavros', 'movie', 'cameron', 'humans', 'world', 'terminator', 'film']
********************
The 5th component:
['way', 'characters', 'character', 'story', 'good', 'time', 'just', 'like', 'movie', 'film']
********************
The 6th component:
['film', 'surgery', 'troi', 'frakes', 'riker', 'irwin', 'trek', 'awakenings', 'borg', 'virgil']
********************
The 7th compon

In [47]:
#Extract feature for each news
nmf_train_features = nmf.transform(doc_train_vec)
nmf_test_features = nmf.transform(doc_test_vec)

In [48]:
lda_train_features = lda.transform(doc_train_vec)
lda_test_features = lda.transform(doc_test_vec)

## 4.分类

这是一个监督学习里面的二元分类问题，分类的方法可以采用传统的统计机器学习方法比如决策树、朴素贝叶斯、逻辑回归、提升树等；也可以使用多层神经网络。

### 4.1朴素贝叶斯
我们首先使用经典的朴素贝叶斯方法进行分类，假设单词的条件概率都是独立的，计算每个类别单词出现的条件概率，每个文本的条件概率就是每个单词的条件概率之积。

In [49]:
import time
from sklearn.metrics import f1_score
from sklearn.naive_bayes import MultinomialNB

In [50]:
def train_test(clf, train_features, test_features):
    start = time.time()
    #训练
    clf.fit(train_features, label_train)
    end = time.time()
    print('训练时间：{:.3f}'.format(end-start))
    #预测
    preds = clf.predict(test_features)
    #计算准准确率
    #Micro F1
    print('Micro F1: {:.3f}'.format(f1_score(label_test, preds, average='micro')))

In [51]:
clf = MultinomialNB()
train_test(clf, nmf_train_features, nmf_test_features)

训练时间：0.003
Micro F1: 0.680


In [52]:
clf = MultinomialNB()
train_test(clf,lda_train_features, lda_test_features)

训练时间：0.002
Micro F1: 0.497


可以看到负面评价中有Bad， don't， doesn't等单词，而正面评价中有good, best, great, love等单词。

### 4.2 集成方法
还可以尝试集成方法，三个臭皮匠赛过诸葛亮。

In [53]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
#clf_ad = AdaBoostClassifier(n_estimators=100, learning_rate=2).fit(X_train_tfidf, y_train)
clf_rf = RandomForestClassifier(n_estimators=50, n_jobs=4)
train_test(clf_rf, nmf_train_features, nmf_test_features)

训练时间：0.164
Micro F1: 0.698


In [54]:
train_test(clf_rf, lda_train_features,lda_test_features)

训练时间：0.159
Micro F1: 0.515


使用默认参数的继承方法效果并不明显。

### 4.3多层神经网络

In [61]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(80, 20), max_iter=500, alpha=1e-4,
                    solver='sgd', verbose=False, tol=1e-5, random_state=1,
                    learning_rate_init=.01)
train_test(mlp, nmf_train_features, nmf_test_features)

训练时间：1.035
Micro F1: 0.713


看上去似乎主题模型分类效果不如词袋模型，很可能在矩阵分解中丢失了部分信息。